## Tuning MNIST Example with Ray

In [1]:
import argparse
import os

from filelock import FileLock
from tensorflow.keras.datasets import mnist
import tensorflow as tf

In [ ]:
#!pip install -q -U ray[default]

In [ ]:
#!pip install tensorboardx

In [2]:
import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.integration.keras import TuneReportCallback

In [3]:
# load notebook plugin for tensorboard
%load_ext tensorboard

In [4]:
def train_mnist(config):
    # https://github.com/tensorflow/tensorflow/issues/32159
    batch_size = 128
    num_classes = 10
    epochs = 12

    with FileLock(os.path.expanduser("~/.data.lock")):
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(config["hidden"], activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(num_classes, activation="softmax")
    ])

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.SGD(
            lr=config["lr"], momentum=config["momentum"]),
        metrics=["accuracy"])

    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=0,
        validation_data=(x_test, y_test),
        callbacks=[TuneReportCallback({
            "mean_accuracy": "accuracy"
        })])

In [5]:
def tune_mnist(num_training_iterations):
    sched = AsyncHyperBandScheduler(time_attr="training_iteration", max_t=400, grace_period=20)

    analysis = tune.run(
        train_mnist,
        name="exp",
        scheduler=sched,
        metric="mean_accuracy",
        mode="max",
        stop={
            "mean_accuracy": 0.99,
            "training_iteration": num_training_iterations
        },
        num_samples=10,
        resources_per_trial={
            "cpu": 1,
            "gpu": 0
        },
        config={
            "threads": 2,
            "lr": tune.uniform(0.001, 0.1),
            "momentum": tune.uniform(0.1, 0.9),
            "hidden": tune.randint(32, 512),
        })
    print("Best hyperparameters found were: ", analysis.best_config)

In [6]:
parser = argparse.ArgumentParser()
parser.add_argument("--smoke-test", action="store_true", help="Finish quickly for testing")

_StoreTrueAction(option_strings=['--smoke-test'], dest='smoke_test', nargs=0, const=True, default=False, type=None, choices=None, help='Finish quickly for testing', metavar=None)

In [7]:
parser.add_argument(
    "--server-address",
    type=str,
    default=None,
    required=False,
    help="The address of server to connect to if using "
    "Ray Client.")

_StoreAction(option_strings=['--server-address'], dest='server_address', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help='The address of server to connect to if using Ray Client.', metavar=None)

In [9]:
args, _ = parser.parse_known_args()

In [10]:
if args.smoke_test:
    ray.init(num_cpus=1, log_to_driver=False)
elif args.server_address:
    ray.util.connect(args.server_address)

In [11]:
#ray.init(num_cpus=1 if args.smoke_test else None)

In [12]:
tune_mnist(num_training_iterations=5 if args.smoke_test else 300)

2021-06-22 09:57:45,398	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265
2021-06-22 09:58:00,380	WARNING function_runner.py:545 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,hidden,lr,momentum
train_mnist_00449_00000,PENDING,,147,0.0523884,0.376506
train_mnist_00449_00001,PENDING,,212,0.0120575,0.485656
train_mnist_00449_00002,PENDING,,164,0.0479928,0.614752
train_mnist_00449_00003,PENDING,,377,0.0343789,0.275421
train_mnist_00449_00004,PENDING,,219,0.0593536,0.375641
train_mnist_00449_00005,PENDING,,217,0.0809285,0.705893
train_mnist_00449_00006,PENDING,,186,0.0480733,0.354352
train_mnist_00449_00007,PENDING,,383,0.0666019,0.330432
train_mnist_00449_00008,PENDING,,451,0.0748031,0.529788
train_mnist_00449_00009,PENDING,,354,0.0645078,0.797033


2021-06-22 09:58:00,893	ERROR syncer.py:72 -- Log sync requires rsync to be installed.
(pid=28572) 2021-06-22 09:58:08.862604: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX AVX2
(pid=28572) 2021-06-22 09:58:08.866508: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 12. Tune using inter_op_parallelism_threads for best performance.
(pid=21760) 2021-06-22 09:58:09.111483: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX AVX2
(pid=21760) 2021-06-22 09:58:09.116386: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 12. Tune using inter_op_parallelism_threads for best performance.
(pid=27256) 2021-06-22 09:58:09.427142: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU support

Result for train_mnist_00449_00000:
  date: 2021-06-22_09-58-11
  done: false
  experiment_id: f8f15716b1354f599abfe1036184c038
  hostname: WGC110DSQBTT2
  iterations_since_restore: 1
  mean_accuracy: 0.8468000292778015
  node_ip: 19.51.28.198
  pid: 28572
  time_since_restore: 3.1949965953826904
  time_this_iter_s: 3.1949965953826904
  time_total_s: 3.1949965953826904
  timestamp: 1624381091
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 00449_00000
  


Trial name,status,loc,hidden,lr,momentum,acc,iter,total time (s)
train_mnist_00449_00000,RUNNING,19.51.28.198:28572,147,0.0523884,0.376506,0.8468,1,3.195
train_mnist_00449_00001,RUNNING,,212,0.0120575,0.485656,,,
train_mnist_00449_00002,RUNNING,,164,0.0479928,0.614752,,,
train_mnist_00449_00003,RUNNING,,377,0.0343789,0.275421,,,
train_mnist_00449_00004,RUNNING,,219,0.0593536,0.375641,,,
train_mnist_00449_00005,RUNNING,,217,0.0809285,0.705893,,,
train_mnist_00449_00006,RUNNING,,186,0.0480733,0.354352,,,
train_mnist_00449_00007,RUNNING,,383,0.0666019,0.330432,,,
train_mnist_00449_00008,RUNNING,,451,0.0748031,0.529788,,,
train_mnist_00449_00009,RUNNING,,354,0.0645078,0.797033,,,


...

(pid=25044) Windows fatal exception: access violation
(pid=25044) 


Result for train_mnist_00449_00007:
  date: 2021-06-22_09-59-39
  done: false
  experiment_id: 2d977abf8650487e84eb500ada591fae
  hostname: WGC110DSQBTT2
  iterations_since_restore: 12
  mean_accuracy: 0.9767500162124634
  node_ip: 19.51.28.198
  pid: 16360
  time_since_restore: 91.03321552276611
  time_this_iter_s: 2.771385669708252
  time_total_s: 91.03321552276611
  timestamp: 1624381179
  timesteps_since_restore: 0
  training_iteration: 12
  trial_id: 00449_00007
  
Result for train_mnist_00449_00007:
  date: 2021-06-22_09-59-39
  done: true
  experiment_id: 2d977abf8650487e84eb500ada591fae
  experiment_tag: 7_hidden=383,lr=0.066602,momentum=0.33043
  hostname: WGC110DSQBTT2
  iterations_since_restore: 12
  mean_accuracy: 0.9767500162124634
  node_ip: 19.51.28.198
  pid: 16360
  time_since_restore: 91.03321552276611
  time_this_iter_s: 2.771385669708252
  time_total_s: 91.03321552276611
  timestamp: 1624381179
  timesteps_since_restore: 0
  training_iteration: 12
  trial_id: 00449_

Trial name,status,loc,hidden,lr,momentum,acc,iter,total time (s)
train_mnist_00449_00000,TERMINATED,,147,0.0523884,0.376506,0.967383,12,62.0723
train_mnist_00449_00001,TERMINATED,,212,0.0120575,0.485656,0.941667,12,68.8417
train_mnist_00449_00002,TERMINATED,,164,0.0479928,0.614752,0.97445,12,74.1806
train_mnist_00449_00003,TERMINATED,,377,0.0343789,0.275421,0.962267,12,81.9032
train_mnist_00449_00004,TERMINATED,,219,0.0593536,0.375641,0.973217,12,74.2125
train_mnist_00449_00005,TERMINATED,,217,0.0809285,0.705893,0.98435,12,78.1306
train_mnist_00449_00006,TERMINATED,,186,0.0480733,0.354352,0.966917,12,78.2352
train_mnist_00449_00007,TERMINATED,,383,0.0666019,0.330432,0.97675,12,91.0332
train_mnist_00449_00008,TERMINATED,,451,0.0748031,0.529788,0.983933,12,90.4226
train_mnist_00449_00009,TERMINATED,,354,0.0645078,0.797033,0.989267,12,89.5127


(pid=16360) Windows fatal exception: access violation
(pid=16360) 
2021-06-22 09:59:39,996	INFO tune.py:549 -- Total run time: 99.63 seconds (99.41 seconds for the tuning loop).


Best hyperparameters found were:  {'threads': 2, 'lr': 0.06450781207799387, 'momentum': 0.7970329276268132, 'hidden': 354}


In [13]:
tensorboard --logdir ~/ray_results

Reusing TensorBoard on port 6006 (pid 15228), started 21:29:08 ago. (Use '!kill 15228' to kill it.)

2021-06-22 19:19:02,243	ERROR worker.py:1116 -- listen_error_messages_raylet: Error while reading from socket: (10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
2021-06-22 19:19:02,247	ERROR worker.py:456 -- print_logs: Error while reading from socket: (10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
2021-06-22 19:19:02,247	ERROR import_thread.py:88 -- ImportThread: Error while reading from socket: (10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
